In [11]:
import dask
import dask.dataframe as dd
import dask.array as da
from dask.delayed import delayed
import numpy as np
import pandas as pd
from dask.threaded import get
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

In [12]:
def readProcessData(filename):
    file_data = pd.read_csv("dataset_final.csv")
    file_data = file_data.drop(["user_id","max_rating"],axis=1)
    file_data = file_data.drop(["country"],axis=1)
    file_data = file_data.drop(["problem_id"],axis=1)
    enc = LabelEncoder()
    enc.fit(file_data['level_type'])
    file_data['level_type'] = enc.transform(file_data['level_type'])
    enc.fit(file_data['rank'])
    file_data['rank'] = enc.transform(file_data['rank'])
    df = file_data
    df_attempt = df["attempts_range"]
    df = df.drop("attempts_range",axis=1)
    df.insert(0,"attempts_range",df_attempt)
    return df

In [13]:
def trainAndTestModel(df):
    X = df.iloc[:,1:]
    y = df.iloc[:,:1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    model = SVC()
    model.fit(X_train,y_train)
    model.score(X_test,y_test)
    return model

In [14]:
def saveModel(model):
    pkl_filename = "pickle_model.pkl"
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)
    with open(pkl_filename, 'rb') as file:
        pickle_model = pickle.load(file)
    

In [15]:
dsk = {'readProcessData' : (readProcessData,'dataset_final.csv'),
       'trainAndTestModel' : (trainAndTestModel,'readProcessData'),
       'saveModel' : (saveModel,'trainAndTestModel')
      }

In [ ]:
get(dsk,['readProcessData','trainAndTestModel','saveModel'])